In [1]:
!nvidia-smi

Sun Jun 20 23:27:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    28W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


In [3]:
# Kaggle API
!pip install --upgrade --force-reinstall --no-deps kaggle
import json
import os
f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
json_data = json.load(f)
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

     |████████████████████████████████| 61kB 6.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=b451873c670650c2a6c95293a1ba36b3baacb6a10d300c70b1706073042bec66
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


This notebook shows how to fine-tune a BERT model (from huggingface) for our dataset recognition task.

Note that internet is needed during the training phase (for downloading the bert-base-cased model). Internet can be turned off during prediction.

In [4]:
# CFG
create_dataset = False

## Install packages

In [ ]:
#!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install datasets --no-index --find-links=file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets

Looking in links: file:///content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/fsspec-0.8.7-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
Processing /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl


In [ ]:
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /content/drive/My Drive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl
     |████████████████████████████████| 901kB 4.3MB/s 


# Import

In [5]:
import re
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc

random.seed(123)
np.random.seed(456)

In [ ]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
#!cp /kaggle/input/coleridge-packages/my_seqeval.py ./
!cp ../input/coleridge-packages/my_seqeval.py ./

# Hyper-parameters

In [6]:
MAX_LENGTH = 64 #512 #64 # max no. words for each sentence.
OVERLAP = 20 #170 #20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Load data

In [ ]:
#papers = {}
#for paper_id in train['Id'].unique():
#    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
#        paper = json.load(f)
#        papers[paper_id] = paper

#!cp -r {paper_train_folder} /content


# Download kagglenb007 output files to local ../input folder
dname = "kagglenb007-get-text"
!mkdir ../input/{dname}
!kaggle kernels output riow1983/{dname} -p ../input/{dname}

mkdir: cannot create directory ‘../input/kagglenb007-get-text’: File exists
Output file downloaded to ../input/kagglenb007-get-text/folds_pubcat.pkl
Kernel log downloaded to ../input/kagglenb007-get-text/kagglenb007-get-text.log 


In [ ]:
folds = pd.read_pickle("../input/kagglenb007-get-text/folds_pubcat.pkl")
folds.drop_duplicates(subset=["Id"], keep='first', ignore_index=True, inplace=True)
papers = {k:v for k,v in zip(folds["Id"], folds["text"])}


#train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
#paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'
#train = pd.read_csv(train_path)
folds = folds[:MAX_SAMPLE]
print(f'No. raw folds rows: {len(folds)}')

# train = train.groupby('Id').agg({
#     'pub_title': 'first',
#     'dataset_title': '|'.join,
#     'dataset_label': '|'.join,
#     'cleaned_label': '|'.join
# }).reset_index()
#print(f'No. grouped training rows: {len(train)}')

No. raw folds rows: 14271


# Transform data to NER format

In [7]:
dname = "nb005-pytorch-bert-for-ner"
#!rm -r {dname}
#!mkdir {dname}

In [ ]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels, verbose=False): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        if verbose:
            print("len(nes): ", len(nes))
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [ ]:
#papers['d0fa7568-7d8e-4db9-870f-f9c6f668c17b']

In [ ]:
def get_ner_json(df, fold, train_or_valid, verbose=False):
    """
    Args:
        df: pd.DataFrame
        fold: int
        train_or_valid: str
        verbose: bool
    """
    assert train_or_valid in ["train", "valid"], "`train_or_valid` must be either 'train' or 'valid'."
    
    if verbose:
        cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
    
    ner_data = []
    pbar = tqdm(total=len(df))
    for i, id, dataset_label in df[['Id', 'dataset_label']].itertuples():
        # paper
        paper = papers[id]
        
        # labels
        #labels = dataset_label.split('|')
        labels = dataset_label
        labels = [clean_training_text(label) for label in labels]
        
        # sentences
        sentences = set([clean_training_text(sentence) for section in paper 
                    for sentence in section['text'].split('.') 
                    ])
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
        
        # positive sample
        for sentence in sentences:
            is_positive, tags = tag_sentence(sentence, labels, verbose=verbose)
            if is_positive:
                if verbose:
                    cnt_pos += 1
                ner_data.append(tags)
            elif any(word in sentence.lower() for word in ['data', 'study']): 
                ner_data.append(tags)
                if verbose:
                    cnt_neg += 1
        
        # process bar
        pbar.update(1)
        if verbose:
            pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

    # shuffling
    random.shuffle(ner_data)


    # write data to file.
    with open(f'./{dname}/fold_{fold}_{train_or_valid}_ner.json', 'w') as f:
        for row in ner_data:
            words, nes = list(zip(*row))
            row_json = {'tokens' : words, 'tags' : nes}
            json.dump(row_json, f)
            f.write('\n')

In [ ]:
if create_dataset:
    # get _ner.json files
    for fold in range(5):
        train = folds[folds["fold"]!=fold+1].reset_index(drop=True)
        valid = folds[folds["fold"]==fold+1].reset_index(drop=True)
        get_ner_json(train, fold+1, "train", verbose=False)
        get_ner_json(valid, fold+1, "valid", verbose=False)

# Fine-tune a BERT model for NER

In [ ]:
#os.environ["LOAD_BEST_MODEL_AT_END"] = "true"
#!echo $LOAD_BEST_MODEL_AT_END

In [ ]:
def train(fold=None, resume_training=False, num_checkpoint=None):
    """
    Args:
        fold: int
        resume_training: bool
        num_checkpoint: int
    """
    dname_child = f"fold_{fold}"

    # foldごとにKaggle Datasetにuploadするためフォルダ名にfold数を入れる
    os.environ["OUTPUT_DIR"] = dname_fold+"/"+dname_child

    # train_ner.json, valid_ner.jsonは本家{dname}フォルダのものを共有
    os.environ["TRAIN_FILE"] = f'./{dname}/fold_{fold}_train_ner.json'
    os.environ["VALID_FILE"] = f'./{dname}/fold_{fold}_valid_ner.json'

    if resume_training:
        if os.path.exists(f"./{dname}/{dname_child}"):
            pass
        else:
            raise KeyError(f"./{dname}/{dname_child} does not exist! Please consider to train w/ resume_training=False.")
        os.environ["MODEL_PATH"] = f"./{dname}/{dname_child}/checkpoint-{num_checkpoint}"
        
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path $MODEL_PATH \
        --train_file $TRAIN_FILE \
        --validation_file $VALID_FILE \
        --num_train_epochs 1 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_strategy 'steps' \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train \
        --do_eval \
        --evaluation_strategy 'steps' \
        --load_best_model_at_end 'true'
    else:
        if os.path.exists(f"./{dname}/{dname_child}"):
            #!rm -r {dname}/{dname_child}
            raise KeyError(f"./{dname}/{dname_child} exists. Please consider to resume training w/ resume_training=True.")
        else:
            !mkdir {dname}/{dname_child}
        
        !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
        --model_name_or_path 'bert-base-cased' \
        --train_file $TRAIN_FILE \
        --validation_file $VALID_FILE \
        --num_train_epochs 1 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --save_strategy 'steps' \
        --save_steps 15000 \
        --output_dir $OUTPUT_DIR \
        --report_to 'none' \
        --seed 123 \
        --do_train \
        --do_eval \
        --evaluation_strategy 'steps' \
        --load_best_model_at_end 'true'

In [ ]:
#  for fold in range(5):
#      print()
#      print(f"#################################### Start training for fold {fold+1} #################################################")
#      train(fold=fold+1, resume_training=False, num_checkpoint=None)

#train(fold=3, resume_training=False, num_checkpoint=None)
#train(resume_training=True, num_checkpoint=45000)

In [12]:
i = 2 # set fold index to use
dname_fold = dname + f"-fold-{i}"
train(fold=i, resume_training=False, num_checkpoint=None)

After the tuning finishes, we should find our model in './nb005-pytorch-bert-for-ner-fold_{fold}'.

# Upload to Kaggle
./nb005-pytorch-bert-for-ner-fold_{fold}をKaggle datasetsにupload

In [13]:
print(dname_fold)

nb005-pytorch-bert-for-ner-fold-2


In [14]:
# [Init] -----------------------------------------------------------------------------------
# foldごとにKaggle Datasetにuploadするためフォルダ名にfold数を入れる
#!mkdir {dname_fold}
!kaggle datasets init -p {dname_fold}
# referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

with open(f"{dname_fold}/dataset-metadata.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["id"] = f"riow1983/{dname_fold}"
data["title"] = dname_fold

with open(f"{dname_fold}/dataset-metadata.json", "w") as jsonFile:
    json.dump(data, jsonFile)

!kaggle datasets create -p {dname_fold}




# [Update] -----------------------------------------------------------------------------------
#!kaggle datasets version -p {dname_fold} -m "[Update] model.bin file added"

Data package template written to: nb005-pytorch-bert-for-ner-fold-2/dataset-metadata.json
Skipping folder: fold_2; use '--dir-mode' to upload folders
Starting upload for file config.json
100% 801/801 [00:03<00:00, 212B/s]
Upload successful: config.json (801B)
Starting upload for file pytorch_model.bin
100% 411M/411M [00:18<00:00, 23.3MB/s]
Upload successful: pytorch_model.bin (411MB)
Starting upload for file tokenizer_config.json
100% 284/284 [00:04<00:00, 70.4B/s]
Upload successful: tokenizer_config.json (284B)
Starting upload for file special_tokens_map.json
100% 112/112 [00:04<00:00, 22.7B/s]
Upload successful: special_tokens_map.json (112B)
Starting upload for file vocab.txt
100% 208k/208k [00:03<00:00, 63.4kB/s]
Upload successful: vocab.txt (208KB)
Starting upload for file training_args.bin
100% 2.30k/2.30k [00:05<00:00, 435B/s]
Upload successful: training_args.bin (2KB)
Starting upload for file optimizer.pt
100% 822M/822M [00:32<00:00, 26.9MB/s]
Upload successful: optimizer.pt (8

In [11]:
!kaggle datasets list --mine

ref                                        title                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------  ---------------------------------  -----  -------------------  -------------  ---------  ---------------  
riow1983/localnb001-transformers-ner       localnb001-transformers-ner          9GB  2021-06-05 14:48:02              0          0  0.375            
ti110106/spacy-cv-4-model                  SpaCy_cv_4_model                   868MB  2021-06-20 02:26:22              0          0  0.1875           
riow1983/nb009-cv                          nb009-cv                           876KB  2021-06-17 23:10:56              2          0  0.125            
ti110106/spacy-train-set                   SPACY_TRAIN_SET                      4GB  2021-06-20 11:29:11              2          0  0.125            
riow1983/nb005-pytorch-bert-for-ner        nb005-pytorch-bert-for-ner         599MB  2021-06-19 05:0

In [ ]:
!kaggle datasets create -p {dname_fold}

Skipping folder: fold_2; use '--dir-mode' to upload folders
Starting upload for file config.json
100% 801/801 [00:12<00:00, 65.6B/s]
Upload successful: config.json (801B)
Starting upload for file pytorch_model.bin
100% 411M/411M [00:31<00:00, 13.8MB/s]
Upload successful: pytorch_model.bin (411MB)
Starting upload for file tokenizer_config.json
100% 284/284 [00:08<00:00, 34.3B/s]
Upload successful: tokenizer_config.json (284B)
Starting upload for file special_tokens_map.json
100% 112/112 [00:06<00:00, 17.2B/s]
Upload successful: special_tokens_map.json (112B)
Starting upload for file vocab.txt
100% 208k/208k [00:09<00:00, 21.5kB/s]
Upload successful: vocab.txt (208KB)
Starting upload for file training_args.bin
100% 2.30k/2.30k [00:06<00:00, 338B/s]
Upload successful: training_args.bin (2KB)
Starting upload for file optimizer.pt
100% 822M/822M [00:54<00:00, 15.7MB/s]
Upload successful: optimizer.pt (822MB)
Starting upload for file scheduler.pt
100% 623/623 [00:10<00:00, 58.9B/s]
Upload su

In [ ]:
# Copy train_ner.json
#!cp train_ner.json ./{dname}/train_ner.json
#!kaggle datasets version -p {dname} -m "[Add] train_ner.json"

Skipping folder: checkpoint-15000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-30000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-45000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-60000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-75000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-90000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-105000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-120000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-135000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-150000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-165000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-180000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-195000; use '--dir-mode' to upload folders
Skipping folder: checkpoint-210000; use '--dir-mode' to upload folders
Skipping fol